In [ ]:
#import statements

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np

In [ ]:
#downloading historical stock data

df_alibaba_closing_prices = yf.download('BABA','2018-11-01','2019-11-01')
df_snap_closing_prices = yf.download('SNAP','2019-01-01','2019-12-31')
df_tesla_closing_prices = yf.download('TSLA','2019-01-01','2019-12-31')

In [ ]:
#creating a list of stock in our portfolio with the info as ['stock name','Long/Short','Date purchased','number of shares']

portfolio_list = [['BABA', 'L', '2018-11-30', 2000],['SNAP', 'L', '2019-03-14', 5000],['TSLA', 'S', '2019-02-01', 1500]]

#sorting the portfolio by purchase date
portfolio_list.sort(key = lambda x: x[2])
print(portfolio_list)

In [ ]:
#creating new dataframes to store our portfolio data

df_portfolio_alibaba = pd.DataFrame()
df_portfolio_snap = pd.DataFrame()
df_portfolio_tsla = pd.DataFrame()

In [ ]:
#converting Date purchased element is list to date format

len_portfolio = len(portfolio_list)
for i in range(len_portfolio):
    portfolio_list[i][2] = datetime.strptime(portfolio_list[i][2], '%Y-%m-%d').date()

In [ ]:
#function to collect data from historical data for our portfolio dataframes

def append_ranges(df_portfolio, closing_prices_df , len_pf, pf_list, i):
    if i < len_pf:
        df_portfolio = pd.concat([df_portfolio, closing_prices_df.loc[pf_list[i][2] : pf_list[len_pf-1][2]]]).drop_duplicates()
        return df_portfolio
        
    if i == len_pf:
        date = pf_list[len_pf][2]
        date = date + timedelta(days=1)
        df_portfolio = pd.concat([df_portfolio, closing_prices_df.loc[pf_list[i][2] : date]]).drop_duplicates()
        return df_portfolio

#for loop for collecting data using the portfolio list and append_ranges

for i in range(len_portfolio):
    if portfolio_list[i][0] == 'BABA':
        df_portfolio_alibaba = append_ranges(df_portfolio_alibaba, df_alibaba_closing_prices, len_portfolio, portfolio_list, i)
        
    elif portfolio_list[i][0] == 'TSLA':
        df_portfolio_tsla = append_ranges(df_portfolio_tsla, df_tesla_closing_prices, len_portfolio, portfolio_list, i)
        
    elif portfolio_list[i][0] == 'SNAP':
        df_portfolio_snap = append_ranges(df_portfolio_snap, df_snap_closing_prices, len_portfolio, portfolio_list, i)

        
#print(df_portfolio_alibaba)
#print(df_portfolio_tsla)
#print(df_portfolio_snap)
#df_portfolio_tsla.info()
#df_portfolio_snap.info()

In [ ]:
#df_portfolio['Returns'] = df_portfolio.Close.pct_change()*100

#calculating returns based on number of shares purchased
for i in range(len_portfolio):
    if portfolio_list[i][0] == 'BABA':
        df_portfolio_alibaba['Returns'] = df_portfolio_alibaba.Close.pct_change() * 100 * portfolio_list[i][3]
        
    elif portfolio_list[i][0] == 'TSLA':
        df_portfolio_tsla['Returns'] = df_portfolio_tsla.Close.pct_change() * 100 * portfolio_list[i][3]
        
    elif portfolio_list[i][0] == 'SNAP':
        df_portfolio_snap['Returns'] = df_portfolio_snap.Close.pct_change() * 100 * portfolio_list[i][3]

#replacing NA with 0
df_portfolio_alibaba['Returns'] = df_portfolio_alibaba['Returns'].fillna(0)
df_portfolio_tsla['Returns'] = df_portfolio_tsla['Returns'].fillna(0)
df_portfolio_snap['Returns'] = df_portfolio_snap['Returns'].fillna(0)

'''print(df_portfolio_alibaba)
print(df_portfolio_tsla)
print(df_portfolio_snap)'''

In [ ]:
#net return for each portfolio and total

TeslaTotalReturn = df_portfolio_tsla['Returns'].sum()
AliBabaTotalReturn = df_portfolio_alibaba['Returns'].sum()
SnapTotalReturn = df_portfolio_snap['Returns'].sum()

NetReturnPlot = pd.concat([df_portfolio_alibaba['Returns'], -df_portfolio_tsla['Returns'], df_portfolio_snap['Returns']]).drop_duplicates() #df_portfolio_alibaba['Returns'] -df_portfolio_tsla['Returns'] + df_portfolio_snap['Returns']
#NetReturnPlot.sort_values(inplace = True)

print(NetReturnPlot.sum())

In [ ]:
#Daily Returns Plot
fig = plt.figure(figsize = (12, 5))
#plot.hist(df_tesla_closing_prices.index, NetReturnPlot, width = 1.5)
NetReturnPlot.plot.hist(bins=100, alpha=0.5)
plt.title('Net Returns')
plt.xlabel('Date')
plt.ylabel('Returns per day')
plt.show(block=True)